# Part 1

--- Day 11: Seating System ---

Your plane lands with plenty of time to spare. The final leg of your journey is a ferry that goes directly to the tropical island where you can finally start your vacation. As you reach the waiting area to board the ferry, you realize you're so early, nobody else has even arrived yet!

By modeling the process people use to choose (or abandon) their seat in the waiting area, you're pretty sure you can predict the best place to sit. You make a quick map of the seat layout (your puzzle input).

The seat layout fits neatly on a grid. Each position is either floor (`.`), an empty seat (`L`), or an occupied seat (`#`). For example, the initial seat layout might look like this:

```
L.LL.LL.LL
LLLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLLL
L.LLLLLL.L
L.LLLLL.LL
```

Now, you just need to model the people who will be arriving shortly. Fortunately, people are entirely predictable and always follow a simple set of rules. All decisions are based on the number of occupied seats adjacent to a given seat (one of the eight positions immediately up, down, left, right, or diagonal from the seat). The following rules are applied to every seat simultaneously:

- If a seat is empty (`L`) and there are no occupied seats adjacent to it, the seat becomes occupied.
- If a seat is occupied (`#`) and four or more seats adjacent to it are also occupied, the seat becomes empty.
- Otherwise, the seat's state does not change.
- Floor (`.`) never changes; seats don't move, and nobody sits on the floor.

After one round of these rules, every seat in the example layout becomes occupied:

```
#.##.##.##
#######.##
#.#.#..#..
####.##.##
#.##.##.##
#.#####.##
..#.#.....
##########
#.######.#
#.#####.##
```

After a second round, the seats with four or more occupied adjacent seats become empty again:

```
#.LL.L#.##
#LLLLLL.L#
L.L.L..L..
#LLL.LL.L#
#.LL.LL.LL
#.LLLL#.##
..L.L.....
#LLLLLLLL#
#.LLLLLL.L
#.#LLLL.##
```

This process continues for three more rounds:

```
#.##.L#.##
#L###LL.L#
L.#.#..#..
#L##.##.L#
#.##.LL.LL
#.###L#.##
..#.#.....
#L######L#
#.LL###L.L
#.#L###.##
```

```
#.#L.L#.##
#LLL#LL.L#
L.L.L..#..
#LLL.##.L#
#.LL.LL.LL
#.LL#L#.##
..L.L.....
#L#LLLL#L#
#.LLLLLL.L
#.#L#L#.##
```

```
#.#L.L#.##
#LLL#LL.L#
L.#.L..#..
#L##.##.L#
#.#L.LL.LL
#.#L#L#.##
..L.L.....
#L#L##L#L#
#.LLLLLL.L
#.#L#L#.##
```

At this point, something interesting happens: the chaos stabilizes and further applications of these rules cause no seats to change state! Once people stop moving around, you count 37 occupied seats.

Simulate your seating area by applying the seating rules repeatedly until no seats change state. How many seats end up occupied?

In [1]:
(import [pathlib [Path]])

In [2]:
(setv INPUT_FILE (/ (.cwd Path) "inputs" "day11" "part1.txt"))

In [3]:
(defn input-lines []
  (with [fp (.open INPUT_FILE)]
    (yield-from (gfor line fp (.strip line)))))

In [4]:
(defn input-grid []
  (tuple (gfor line (input-lines) (tuple line))))

In [5]:
(defn cell@ [grid row col]
  (get (get grid row) col))

In [6]:
(defn adjacent-cells [grid row col]
  (setv row-start (max 0 (dec row)))
  (setv row-end (min (dec (len grid)) (inc row)))
  (setv col-start (max 0 (dec col)))
  (setv col-end (min (dec (len (get grid 0))) (inc col)))
  (for [[r c] (product (range row-start (inc row-end)) (range col-start (inc col-end)))]
    (unless (and (= r row) (= c col))
      (yield (cell@ grid r c)))))

In [7]:
(defn adjacent-occupied [grid row col]
  (sum (gfor cell (adjacent-cells grid row col) (= cell "#"))))

In [8]:
(defn new-value [grid row col]
  (setv cell (cell@ grid row col))
  (cond
    [(= cell "L") (if (= (adjacent-occupied grid row col) 0) "#" "L")]
    [(= cell "#") (if (>= (adjacent-occupied grid row col) 4) "L" "#")]
    [(= cell ".") "."]))

In [9]:
(defn do-cycle [grid]
  (setv rows (len grid))
  (setv cols (len (get grid 0)))
  (tuple
    (gfor
      row
      (range rows)
      (tuple
        (gfor
          col
          (range cols)
          (new-value grid row col))))))

In [10]:
(defn run-model [grid]
  (setv next-grid (do-cycle grid))
  (while (!= grid next-grid)
    (setv [grid next-grid] [next-grid (do-cycle next-grid)]))
  next-grid)

In [11]:
(setv result (do-cycle (run-model (input-grid))))

In [12]:
(sum (gfor char (flatten result) (= char "#")))

2368

# Part 2

As soon as people start to arrive, you realize your mistake. People don't just care about adjacent seats - they care about the first seat they can see in each of those eight directions!

Now, instead of considering just the eight immediately adjacent seats, consider the first seat in each of those eight directions. For example, the empty seat below would see eight occupied seats:

```
.......#.
...#.....
.#.......
.........
..#L....#
....#....
.........
#........
...#.....
```

The leftmost empty seat below would only see one empty seat, but cannot see any of the occupied ones:

```
.............
.L.L.#.#.#.#.
.............
```

The empty seat below would see no occupied seats:

```
.##.##.
#.#.#.#
##...##
...L...
##...##
#.#.#.#
.##.##.
```

Also, people seem to be more tolerant than you expected: it now takes five or more visible occupied seats for an occupied seat to become empty (rather than four or more from the previous rules). The other rules still apply: empty seats that see no occupied seats become occupied, seats matching no rule don't change, and floor never changes.

Given the same starting layout as above, these new rules cause the seating area to shift around as follows:

```
L.LL.LL.LL
LLLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLLL
L.LLLLLL.L
L.LLLLL.LL
```

```
#.##.##.##
#######.##
#.#.#..#..
####.##.##
#.##.##.##
#.#####.##
..#.#.....
##########
#.######.#
#.#####.##
```

```
#.LL.LL.L#
#LLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLL#
#.LLLLLL.L
#.LLLLL.L#
```

```
#.L#.##.L#
#L#####.LL
L.#.#..#..
##L#.##.##
#.##.#L.##
#.#####.#L
..#.#.....
LLL####LL#
#.L#####.L
#.L####.L#
```

```
#.L#.L#.L#
#LLLLLL.LL
L.L.L..#..
##LL.LL.L#
L.LL.LL.L#
#.LLLLL.LL
..L.L.....
LLLLLLLLL#
#.LLLLL#.L
#.L#LL#.L#
```

```
#.L#.L#.L#
#LLLLLL.LL
L.L.L..#..
##L#.#L.L#
L.L#.#L.L#
#.L####.LL
..#.#.....
LLL###LLL#
#.LLLLL#.L
#.L#LL#.L#
```

```
#.L#.L#.L#
#LLLLLL.LL
L.L.L..#..
##L#.#L.L#
L.L#.LL.L#
#.LLLL#.LL
..#.L.....
LLL###LLL#
#.LLLLL#.L
#.L#LL#.L#
```

Again, at this point, people stop shifting around and the seating area reaches equilibrium. Once this occurs, you count 26 occupied seats.

Given the new visibility method and the rule change for occupied seats becoming empty, once equilibrium is reached, how many seats end up occupied?

In [13]:
(defn seat? [char]
  (when (in char ["L" "#"])
    char))

In [14]:
(defn seat-north [grid row col]
  (some
    seat?
    (gfor
      r
      (range (dec row) -1 -1)
      (cell@ grid r col))))

In [15]:
(defn seat-south [grid row col]
  (some
    seat?
    (gfor
      r
      (range (inc row) (len grid))
      (cell@ grid r col))))

In [17]:
(defn seat-west [grid row col]
  (some
    seat?
    (gfor
      c
      (range (dec col) -1 -1)
      (cell@ grid row c))))

In [16]:
(defn seat-east [grid row col]
  (some
    seat?
    (gfor
      c
      (range (inc col) (len (get grid 0)))
      (cell@ grid row c))))

In [19]:
(defn seat-ne [grid row col]
  ; row -
  ; col +
  (some
    seat?
    (gfor
      [r c]
      (zip (range (dec row) -1 -1) (range (inc col) (len (get grid 0))))
      (cell@ grid r c))))

In [18]:
(defn seat-se [grid row col]
  ; row +
  ; col +
  (some
    seat?
    (gfor
      [r c]
      (zip (range (inc row) (len grid)) (range (inc col) (len (get grid 0))))
      (cell@ grid r c))))

In [20]:
(defn seat-sw [grid row col]
  ; row +
  ; col -
  (some
    seat?
    (gfor
      [r c]
      (zip (range (inc row) (len grid)) (range (dec col) -1 -1))
      (cell@ grid r c))))

In [21]:
(defn seat-nw [grid row col]
  ; row -
  ; col -
  (some
    seat?
    (gfor
      [r c]
      (zip (range (dec row) -1 -1) (range (dec col) -1 -1))
      (cell@ grid r c))))

In [22]:
(defn visible-seats [grid row col]
  (yield-from
    (remove
      none? [
      (seat-north grid row col)
      (seat-east grid row col)
      (seat-south grid row col)
      (seat-west grid row col)
      (seat-ne grid row col)
      (seat-se grid row col)
      (seat-sw grid row col)
      (seat-nw grid row col)])))

In [23]:
(defn visible-occupied [grid row col]
  (sum (gfor seat (visible-seats grid row col) (= seat "#"))))

In [24]:
(defn new-value2 [grid row col]
  (setv cell (cell@ grid row col))
  (cond
    [(= cell "L") (if (= (visible-occupied grid row col) 0) "#" "L")]
    [(= cell "#") (if (>= (visible-occupied grid row col) 5) "L" "#")]
    [(= cell ".") "."]))

In [25]:
(list (visible-seats (input-grid) 10 10))

['L', 'L', 'L', 'L', 'L', 'L', 'L', 'L']

In [26]:
(visible-occupied (input-grid) 10 10)

0

In [27]:
(defn do-cycle2 [grid]
  (setv rows (len grid))
  (setv cols (len (get grid 0)))
  (tuple
    (gfor
      row
      (range rows)
      (tuple
        (gfor
          col
          (range cols)
          (new-value2 grid row col))))))

In [28]:
(defn run-model2 [grid]
  (setv next-grid (do-cycle2 grid))
  (while (!= grid next-grid)
    (setv [grid next-grid] [next-grid (do-cycle2 next-grid)]))
  next-grid)

In [29]:
(setv result2 (run-model2 (input-grid)))

In [30]:
(sum (gfor char (flatten result2) (= char "#")))

2124